# Execução do modelo

### Autenticação

In [ ]:
from scripts.authentcation import authenticate_huggingface

authenticate_huggingface()

### Carregamento do dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset('marmal88/skin_cancer', split='test')

### Carregamento do modelo

In [ ]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    'unsloth/Llama-3.2-11B-Vision-Instruct',
    load_in_4bit=True,
    use_gradient_checkpointing='unsloth',
)

### Inferência

In [ ]:
from IPython.display import display
from scripts.messages import add_inference_message

FastVisionModel.for_inference(model)

index = int(input('Select an index: '))
data = dataset[index]
image = data['image']

messages = add_inference_message(input('Prompt: '))

input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

inputs = tokenizer(
    image,
    input_text,
    add_special_tokens=False,
    return_tensors='pt',
).to('cuda')

outputs = model.generate(
    **inputs,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.0,
    min_p=0.1
)

result = tokenizer.decode(outputs[0], skip_special_tokens=True)
assistant_msg = result.split('assistant')[-1].strip()

print(assistant_msg)
print(data)
display(image)